In [9]:
import findspark
findspark.init()

In [75]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("test").getOrCreate()
from pyspark.sql.functions import format_string
import sys
from pyspark import SparkContext
from csv import reader
from pyspark.sql.functions import date_format
from pyspark.sql.functions import col
from datetime import datetime
from pyspark.sql.functions import round

In [59]:
df1 = spark.read.format("csv").options(header = 'true', inferSchema = 'true').load("C:\\Users\jack_\Desktop\Big_data\Trips.csv")
df2 = spark.read.format("csv").options(header = 'true', inferSchema = 'true').load("C:\\Users\jack_\Desktop\Big_data\Fares.csv")
df3 = spark.read.format("csv").options(header = 'true', inferSchema = 'true').load("C:\\Users\jack_\Desktop\Big_data\Licenses.csv")
df1.createOrReplaceTempView("table1")
df2.createOrReplaceTempView("table2")
df3.createOrReplaceTempView("table3")
lines = sc.textFile("C:\\Users\jack_\Desktop\Big_data\Trips.csv", 5)
lines2 = sc.textFile("C:\\Users\jack_\Desktop\Big_data\Fares.csv", 5)
lines3 = sc.textFile("C:\\Users\jack_\Desktop\Big_data\Licenses.csv", 5)
lines = lines.mapPartitions(lambda x: reader(x))
lines2 = lines2.mapPartitions(lambda x: reader(x))
lines3 = lines3.mapPartitions(lambda x: reader(x))
header = lines.take(1)[0]
header2 = lines2.take(1)[0]
header3 = lines3.take(1)[0]
rdd = lines.filter(lambda x: x != header)
rdd2 = lines2.filter(lambda x: x != header2)
rdd3 = lines3.filter(lambda x: x != header3)

[['medallion',
  'hack_license',
  'vendor_id',
  'pickup_datetime',
  'payment_type',
  'fare_amount',
  'surcharge',
  'mta_tax',
  'tip_amount',
  'tolls_amount',
  'total_amount']]

In [53]:
dfFilter = spark.sql("select * from table1 where passenger_count > 0 order by medallion, hack_license, pickup_datetime")
trips = dfFilter.alias('trips')
fares = df2.alias('fares')
inner_join = trips.join(fares, (trips.medallion == fares.medallion) & (trips.hack_license == fares.hack_license) & (trips.vendor_id == fares.vendor_id) & (trips.pickup_datetime == fares.pickup_datetime))
inner_join1 = inner_join.select(trips.medallion, trips.hack_license, trips.pickup_datetime, trips.vendor_id, inner_join.rate_code, inner_join.store_and_fwd_flag, inner_join.dropoff_datetime, inner_join.passenger_count, inner_join.trip_time_in_secs, inner_join.trip_distance, inner_join.pickup_longitude, inner_join.pickup_latitude, inner_join.dropoff_longitude, inner_join.dropoff_latitude, inner_join.payment_type, inner_join.fare_amount, inner_join.surcharge, inner_join.mta_tax, inner_join.tip_amount, inner_join.tolls_amount, inner_join.total_amount)
inner_join1.createOrReplaceTempView('table4')
inner_join1.select(format_string('%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,',inner_join1.medallion, inner_join1.hack_license, inner_join1.pickup_datetime, inner_join1.vendor_id, inner_join1.rate_code, inner_join1.store_and_fwd_flag, inner_join1.dropoff_datetime, inner_join1.passenger_count, inner_join1.trip_time_in_secs, inner_join1.trip_distance, inner_join1.pickup_longitude, inner_join1.pickup_latitude, inner_join1.dropoff_longitude, inner_join1.dropoff_latitude, inner_join1.payment_type, inner_join1.fare_amount, inner_join1.surcharge, inner_join1.mta_tax, inner_join1.tip_amount, inner_join1.tolls_amount, inner_join1.total_amount).alias('result')).write.save("task1a-sql.out", format="text")

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|result                                                                                                                                                                                                                    |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|00005007A9F30E289E760362F69E4EAD, 39C3E34B3E338A7210C1D3163E6E9589, 2013-08-05 14:38:11, CMT, 1, N, 2013-08-05 14:52:44, 1, 873, 2.5, -73.992638, 40.743778, -74.007828, 40.724129, CRD, 12.0, 0.0, 0.5, 1.5, 0.0, 14.0,  |
|000351EDC735C079246435340A54C7C1, 16455C0F0C008C18A457A8DAB98B5715, 2013-08-01 20:04:15, CMT, 1, N, 2013-08-01 20:1

In [66]:
filtered = rdd.filter(lambda x: int(x[7]) > 0)
mapped2 = rdd2.map(lambda y: ((y[0], y[1], y[2], y[3]), (y[4], y[5], y[6], y[7], y[8], y[9], y[10])))
mapped = filtered.map(lambda key: ((key[0], key[1], key[2], key[5]), (key[3], key[4], key[6], key[7], key[8], key[9], key[10], key[11], key[12], key[13])))
inner_join = mapped.join(mapped2)
sorted = inner_join.sortByKey(True)
sorted.saveAsTextFile("task1a.out")

[(('00005007A9F30E289E760362F69E4EAD',
   '2C584442C9DC6740767CDE5672C12379',
   'CMT',
   '2013-08-06 23:44:12'),
  (('1',
    'N',
    '2013-08-06 23:44:53',
    '1',
    '40',
    '0.60000',
    '-73.9626460000',
    '40.8046420000',
    '-73.9592130000',
    '40.8091390000'),
   ('CSH',
    '4.0000000000',
    '0.5000000000',
    '0.5000000000',
    '0.0000000000',
    '0.0000000000',
    '5.0000000000'))),
 (('00005007A9F30E289E760362F69E4EAD',
   '2C584442C9DC6740767CDE5672C12379',
   'CMT',
   '2013-08-06 23:59:31'),
  (('1',
    'N',
    '2013-08-07 00:07:40',
    '3',
    '489',
    '1.50000',
    '-73.9876710000',
    '40.7752760000',
    '-73.9814680000',
    '40.7640690000'),
   ('CSH',
    '7.5000000000',
    '0.5000000000',
    '0.5000000000',
    '0.0000000000',
    '0.0000000000',
    '8.5000000000')))]

In [94]:
sorted = spark.sql("select * from table2 order by medallion, hack_license, pickup_datetime asc")
fares = sorted.alias('fares')
licenses = df3.alias('licenses')
joined = fares.join(licenses, (fares.medallion == licenses.medallion))
inner_join = joined.select(fares.medallion, joined.hack_license, joined.pickup_datetime, joined.vendor_id, joined.payment_type, joined.fare_amount, joined.surcharge, joined.mta_tax, joined.tip_amount, joined.tolls_amount, joined.total_amount, joined.name, joined.type, joined.current_status, joined.DMV_license_plate, joined.vehicle_VIN_number, joined.vehicle_type, joined.model_year, joined.medallion_type, joined.agent_number, joined.agent_name, joined.agent_telephone_number, joined.agent_website, joined.agent_address, joined.last_updated_date, joined.last_updated_time)
inner_join.createOrReplaceTempView('table5')
inner_join.select(format_string('%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s',inner_join.medallion, joined.hack_license, joined.pickup_datetime, joined.vendor_id, joined.payment_type, joined.fare_amount, joined.surcharge, joined.mta_tax, joined.tip_amount, joined.tolls_amount, joined.total_amount, joined.name, joined.type, joined.current_status, joined.DMV_license_plate, joined.vehicle_VIN_number, joined.vehicle_type, joined.model_year, joined.medallion_type, joined.agent_number, joined.agent_name, joined.agent_telephone_number, joined.agent_website, joined.agent_address, joined.last_updated_date, joined.last_updated_time).alias('Result')).write.save("task1b-sql.out", format="text")

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Result                                                                                                                                                                                                                                                                                                                                  |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|AA7023

In [98]:
mapped = rdd2.map(lambda key: ((key[0]), (key[1], key[2], key[3], key[4], key[5], key[6], key[7], key[8], key[9], key[10])))
mapped2 = rdd3.map(lambda y: ((y[0]), (y[1], y[2], y[3], y[4], y[5], y[6], y[7], y[8], y[9], y[10], y[11], y[12], y[13], y[14], y[15])))
joined = mapped.join(mapped2)
sorted1 = joined.sortBy(lambda x: (x[0], x[1][0]), True)
sorted1.saveAsTextFile("task1b.out")

[('00005007A9F30E289E760362F69E4EAD',
  (('2C584442C9DC6740767CDE5672C12379',
    'CMT',
    '2013-08-06 23:44:12',
    'CSH',
    '4.0000000000',
    '0.5000000000',
    '0.5000000000',
    '0.0000000000',
    '0.0000000000',
    '5.0000000000'),
   ('PLANOS, INC.',
    'MEDALLION',
    'CUR',
    '3G31C',
    '2FABP7AV9BX153848',
    'CNG',
    '2011',
    'NAMED DRIVER',
    '304',
    'FA  MANAGEMENT INC.',
    '(718)204-9241',
    '',
    '11-06 BROADWAY ASTORIA NY 11106',
    '1/31/2015',
    '13:20'))),
 ('00005007A9F30E289E760362F69E4EAD',
  (('2C584442C9DC6740767CDE5672C12379',
    'CMT',
    '2013-08-06 23:59:31',
    'CSH',
    '7.5000000000',
    '0.5000000000',
    '0.5000000000',
    '0.0000000000',
    '0.0000000000',
    '8.5000000000'),
   ('PLANOS, INC.',
    'MEDALLION',
    'CUR',
    '3G31C',
    '2FABP7AV9BX153848',
    'CNG',
    '2011',
    'NAMED DRIVER',
    '304',
    'FA  MANAGEMENT INC.',
    '(718)204-9241',
    '',
    '11-06 BROADWAY ASTORIA NY 11106',
 

In [38]:
filtered = spark.sql("select count(table4.medallion) as num_trips from table4 where fare_amount >= 0 and fare_amount <= 5")
filtered1 = spark.sql("select count(table4.medallion) as num_trips from table4 where fare_amount > 5 and fare_amount <= 15")
filtered2 = spark.sql("select count(table4.medallion) as num_trips from table4 where fare_amount > 15 and fare_amount <= 30")
filtered3 = spark.sql("select count(table4.medallion) as num_trips from table4 where fare_amount > 30 and fare_amount <= 50")
filtered4 = spark.sql("select count(table4.medallion) as num_trips from table4 where fare_amount > 50 and fare_amount <= 100")
filtered5 = spark.sql("select count(table4.medallion) as num_trips from table4 where fare_amount > 100")
result = "0, 5, {0} \n 5, 15, {1} \n 15, 30, {2} \n 30, 50, {3} \n 50, 100, {4} \n 100, {5}".format(filtered.collect()[0][0], filtered1.collect()[0][0], filtered2.collect()[0][0], filtered3.collect()[0][0], filtered4.collect()[0][0], filtered5.collect()[0][0])
save = sc.parallelize(result)
save.saveAsTextFile("task2a-sql.out")

0, 5, 273080 
 5, 15, 1531192 
 15, 30, 360200 
 30, 50, 76906 
 50, 100, 50897 
 100, 918


In [46]:
range = sorted.filter(lambda x: (float(x[1][1][1]) >= 0 and float(x[1][1][1]) <= 5))
range1 = sorted.filter(lambda x: (float(x[1][1][1]) > 5 and float(x[1][1][1]) <= 15))
range2 = sorted.filter(lambda x: (float(x[1][1][1]) > 15 and float(x[1][1][1]) <= 30))
range3 = sorted.filter(lambda x: (float(x[1][1][1]) > 30 and float(x[1][1][1]) <= 50))
range4 = sorted.filter(lambda x: (float(x[1][1][1]) > 50 and float(x[1][1][1]) <= 100))
range5 = sorted.filter(lambda x: (float(x[1][1][1]) > 100))
result = "0, 5, {0} \n 5, 15, {1} \n 15, 30, {2} \n 30, 50, {3} \n 50, 100, {4} \n 100, {5}".format(range.count(), range1.count(), range2.count(), range3.count(), range4.count(), range5.count())
save = sc.parallelize(result)
save.saveAsTextFile("task2a.out")

0, 5, 273080 
 5, 15, 1531192 
 15, 30, 360200 
 30, 50, 76906 
 50, 100, 50897 
 100, 918


In [49]:
filtered = spark.sql("select passenger_count, count(medallion ) as num_trips from table4 group by passenger_count order by passenger_count asc")
result = filtered.select(format_string('%s, %s', filtered.passenger_count, filtered.num_trips).alias("Result"))
result.write.save("task2b-sql.out", format="text")

+----------+
|Result    |
+----------+
|1, 1691674|
|2, 309238 |
|3, 97031  |
|4, 53465  |
|5, 83910  |
|6, 58289  |
|8, 1      |
+----------+



In [52]:
mapped = sorted.map(lambda x: ((x[1][0][3]), x[0][0]))
mapped.groupByKey().mapValues(len).sortByKey().saveAsTextFile("task2b.out")

[('1', 1691674),
 ('2', 309238),
 ('3', 97031),
 ('4', 53465),
 ('5', 83910),
 ('6', 58289),
 ('8', 1)]

In [58]:
filtered = spark.sql("select Cast(pickup_datetime as DATE) as fulldate, cast(sum(fare_amount + surcharge + tip_amount) as DECIMAL(9,2)) as total_revenue, cast(sum(tolls_amount) as DECIMAL(8,2)) as total_tolls from table4 group by Cast(pickup_datetime as DATE) order by Cast(pickup_datetime as DATE), total_revenue, total_tolls asc")
new = filtered.select(date_format(col("fulldate"), "MM/dd/YYYY").alias("Date"), filtered.total_revenue, filtered.total_tolls)
new.select(format_string('%s, %s, %s', new.Date, new.total_revenue, new.total_tolls).alias("Result")).write.save("task2c-sql.out", format="text")

+---------------------------------+
|Result                           |
+---------------------------------+
|08/01/2013, 4342019.99, 73060.70 |
|08/02/2013, 3351559.36, 61214.54 |
|08/03/2013, 3027483.82, 41847.17 |
|08/04/2013, 2914129.17, 58840.40 |
|08/05/2013, 5792022.94, 125630.00|
|08/06/2013, 6244012.62, 113876.93|
|08/07/2013, 6573878.89, 123809.01|
+---------------------------------+



In [63]:
mapped = sorted.map(lambda x: ((datetime.strptime(x[0][3], "%Y-%m-%d %H:%M:%S")), (float(x[1][1][1]) + float(x[1][1][2]) + float(x[1][1][4]), float(x[1][1][5]))))
mapped1 = mapped.map(lambda x: ((x[0].strftime("%m/%d/%Y")), (x[1][0], x[1][1])))
result = mapped1.reduceByKey(lambda x, y: (round(x[0] + y[0], 2), round(x[1] + y[1], 2))).sortByKey()
result.saveAsTextFile("task2c.out")

[('08/01/2013', (4342019.99, 73060.7)),
 ('08/02/2013', (3351559.36, 61214.54)),
 ('08/03/2013', (3027483.82, 41847.17)),
 ('08/04/2013', (2914129.17, 58840.4)),
 ('08/05/2013', (5792022.94, 125630.0)),
 ('08/06/2013', (6244012.62, 113876.93)),
 ('08/07/2013', (6573878.89, 123809.01))]

In [64]:
filtered = spark.sql("select medallion, cast(pickup_datetime as DATE) as days_driven, count(pickup_datetime) as total_trips from table4 where passenger_count > 0 group by medallion, days_driven")
filtered.createOrReplaceTempView("table6")
new = spark.sql("select medallion, count(days_driven) as days, sum(total_trips) as totals from table6 group by medallion")
new.createOrReplaceTempView("table7")
result = spark.sql("select medallion, totals as total_trips, days as days_driven, cast(totals/days as DECIMAL(4,2)) as average from table7")
result.select(format_string('%s, %s, %s, %s', result.medallion, result.total_trips, result.days_driven, result.average).alias("result")).write.save("task2d-sql.out", format="text")

+-----------------------------------------------+
|result                                         |
+-----------------------------------------------+
|1E8EDF1C2EF489B7AB3712977C7C08B5, 123, 4, 30.75|
|B2B089B939CB4A0A64D42CBFC74059BD, 163, 4, 40.75|
|4E8142153D6520C415A644671066AA97, 86, 4, 21.50 |
|9B69C5971F62F151BB1C412B35090015, 253, 7, 36.14|
|47D63452A91E1705FE73C7510F1DDCF7, 307, 7, 43.86|
|18DDC049CC26AE3AABB4C08C59374EFB, 45, 4, 11.25 |
|7D4F34EF0A251F3A6A8DF3CE4A098B8C, 181, 7, 25.86|
|C0D5941A4A93777E96210EBD9DCAEC95, 195, 7, 27.86|
|618BB39CEEAE5E9A6D886E20E005DCA2, 79, 4, 19.75 |
|72EAFBA3FB9F0507C671AD713A622FB6, 115, 4, 28.75|
|27E7626D5A223B47957D82B66E7BD262, 159, 7, 22.71|
|15162141EA7436635C696F5BC023D2D6, 258, 7, 36.86|
|59DF6039EC312EE6DAE98E8DF115A679, 138, 4, 34.50|
|DB1964B903773868E191176E8EF47946, 86, 3, 28.67 |
|35B2F21FAF5E53F1EB17848E7DC82055, 203, 7, 29.00|
|E7C49B0A85D992BF1F5659C160B7FD7F, 234, 7, 33.43|
|496036713FC662D7165AF3CE796FCAAA, 79, 4, 19.75 |


In [67]:
filtered = sorted.map(lambda x: ((x[0][0]), x[0][3]))
mapped = filtered.groupByKey().mapValues(len).sortByKey()
new = filtered.map(lambda x: ((x[0]), datetime.strptime(x[1], "%Y-%m-%d %H:%M:%S")))
filtered2 = new.map(lambda x: ((x[0], x[1].strftime("%d/%m/%Y")), 1))
tmp = filtered2.reduceByKey(lambda x, y: x + y)
mapped2 = tmp.map(lambda x: ((x[0][0]), x[0][1])).groupByKey().mapValues(len)
joined = mapped.join(mapped2)
result = joined.map(lambda x: (x[0], x[1][0], x[1][1], round(float(x[1][0])/float(x[1][1]), 2)))
result.saveAsTextFile("task2d.out")

[('049AA6C216DCA918917AB24189DAF99D', 280, 7, 40.0),
 ('0654DD90E7D0F7F266837C94CCB65825', 150, 6, 25.0),
 ('08E54F4C460720DDE43460E354486FBC', 195, 7, 27.86),
 ('091334CDF9A2292C04B6AF20AC219660', 105, 4, 26.25),
 ('0923CB4C0071D475AB0B4519B8B2F4F8', 310, 7, 44.29),
 ('096ACCFC956CC07673EC193B24DAAA3E', 113, 4, 28.25),
 ('0B10381B3ADC2120244F34E1FEA0A008', 236, 7, 33.71),
 ('0B84045AE4809A95A32F796400C8B403', 138, 4, 34.5),
 ('0BD37C7056571AE5F89306C78A733C79', 59, 3, 19.67),
 ('0BECD68D5362BE504DFC27A0F914DDAA', 247, 7, 35.29)]

In [68]:
filtered = spark.sql("select count(pickup_datetime) as num_invalid_trips from table4 where fare_amount < 0")
filteredwrite.save("task3a-sql.out", format="text")

+-----------------+
|num_invalid_trips|
+-----------------+
|              415|
+-----------------+



In [70]:
filtered = sorted.filter(lambda x: float(x[1][1][1]) < 0).map(lambda x: x[0][3])
result = filtered.map(lambda x: 1).reduce(lambda x, y: x+y)
save = sc.parallelize(result)
save.saveAsTextFile("task3a.out")

415


In [71]:
filtered = spark.sql("select medallion, pickup_datetime, count(*) as repeats from table4 group by medallion, pickup_datetime having count(*) > 1")
result = filtered.select(format_string('%s, %s, %s', filtered.medallion, filtered.pickup_datetime, filtered.repeats).alias("Result"))
result.write.save("task3b-sql.out", format="text")

+--------------------------------------------------------+
|Result                                                  |
+--------------------------------------------------------+
|A2CF7F240874DF15FE3EB68414ABF1DC, 2013-08-07 16:08:00, 4|
|351E1091AB0ADB554152A55BBA66E4ED, 2013-08-06 09:53:00, 4|
|A44E3AAB61400E932DAF086634E19685, 2013-08-05 18:50:00, 4|
|BC3A3D1BECC84F0FAAEAC0F725FC221E, 2013-08-01 06:31:00, 4|
|DBA03C39ED8C744C7624D3E40E88893D, 2013-08-07 22:41:00, 4|
|954098685214C4B5150E5CC130812936, 2013-08-07 19:02:00, 4|
|230FBEE57EBAF0D85F5F13DFDD66B181, 2013-08-05 21:13:00, 4|
|A2CF7F240874DF15FE3EB68414ABF1DC, 2013-08-07 12:09:00, 4|
|F23DDD7B961109E4EB194F3F7AF20F7D, 2013-08-05 15:32:00, 4|
|2B8A6A227F262737D21F9E93B2FA0D65, 2013-08-05 15:49:00, 4|
|7219E4C7C68C04DCCCFBA39D793AC641, 2013-08-06 15:22:00, 4|
|EE9FB7759494B99129A19BE5058F94B8, 2013-08-05 06:11:00, 4|
|EBED0E655DDFA52D3F86B7B0D9CD41C5, 2013-08-07 01:08:00, 4|
|77EC6F21C85B637CC42BB997841E11A6, 2013-08-05 15:15:00, 

In [73]:
mapped = sorted.map(lambda x: ((x[0][0], x[0][3]), x[1][1][1]))
reduced = mapped.map(lambda x: ((x[0][0], x[0][1]), 1)).reduceByKey(lambda x, y: x+y)
result = reduced.filter(lambda x: x[1] > 1)
result.saveAsTextFile("task3b.out")

[(('5493A58B275E12A44971656845F86B93', '2013-08-05 21:06:00'), 4),
 (('5F799475358AF4726C2178539F24A91C', '2013-08-06 17:31:00'), 2),
 (('696A7ECC869379ED31033CC897A42BFC', '2013-08-05 05:39:00'), 4),
 (('6D9CCBF092BDC6CA596C54FA13C98B79', '2013-08-05 14:02:00'), 4),
 (('7B501672204EB6B8777CE5C16296AC66', '2013-08-03 23:11:10'), 4),
 (('94E81376D0E03B9A4CD02D98728AAFD2', '2013-08-06 18:11:00'), 4),
 (('954098685214C4B5150E5CC130812936', '2013-08-07 19:02:00'), 4),
 (('A2CF7F240874DF15FE3EB68414ABF1DC', '2013-08-07 17:07:00'), 4),
 (('CE22FDCE2D5845CF3E562173084FFE99', '2013-08-05 18:15:00'), 4),
 (('D46130EEE58952844E13AE96E6A44362', '2013-08-05 21:15:00'), 4)]

In [76]:
filtered = spark.sql("select medallion, count(pickup_datetime) as total_trips from table4 where passenger_count > 0 group by medallion")
filtered2 = spark.sql("select medallion, count(pickup_datetime) as errors from table4 where pickup_latitude = 0 and pickup_longitude = 0 and dropoff_latitude = 0 and dropoff_longitude = 0 group by medallion")
joined = filtered.join(filtered2, filtered.medallion == filtered2.medallion)
result = joined.select(filtered.medallion, ((joined.errors/joined.total_trips)*100).alias('percentage'))
rounded = result.select(col("medallion"), round(col("percentage"), 2).alias("percentage"))
output = rounded.select(format_string('%s, %s%%', rounded.medallion, rounded.percentage).alias("result"))
output.write.save("task3c-sql.out", format="text")

+----------------------------------------+
|result                                  |
+----------------------------------------+
|18DDC049CC26AE3AABB4C08C59374EFB, 4.44% |
|35E11D9D2AE5C8A80261CF6A309BD9FD, 45.55%|
|4E8142153D6520C415A644671066AA97, 97.67%|
|5CCB4924B158F945B1792BCD0FB16F5C, 1.3%  |
|753BC0484097BB236807E896DC2241DF, 1.89% |
|7550D0BD520A691EC058F3C44FCEE2C0, 0.93% |
|963BEE5F306952D20A2DB2BE7B166205, 0.61% |
|A02946A94C960AF041A251269D57C0B4, 0.63% |
|A8608B0C1235FCEB98A71197646FB3D9, 3.33% |
|ADFCF211DDD6D788567104FF7B605626, 6.86% |
|EB6F0753E865DA0ABA7B249F4B8CF51E, 1.05% |
|ED9B774735449ABBE4FE4062765C21CF, 2.27% |
|EF882BDAF03D4151746F1A5A235FC454, 2.63% |
|38369565598C05AD61BFE7E83BF72382, 1.05% |
|552CCF061B871F7179CC0357086CB42C, 1.09% |
|D50F5974294A3AC413B8291806D46433, 2.6%  |
|F0581E1A2A56A5917506660C8FFA7878, 1.18% |
|03F2B14F5C52CC104530F0C895AA750C, 0.96% |
|1087DFA00B5B8DBBDEFD0D32D5E8E0C3, 4.41% |
|14B6352E4AC7C4732A518085AF4253DD, 4.55% |
+----------

In [89]:
filtered = sorted.map(lambda x: ((x[0][0]), x[0][3]))
reduced = filtered.map(lambda x: ((x[0]), 1)).reduceByKey(lambda x, y: x+y)
filtered1 = sorted.filter(lambda x: float(x[1][0][6]) == 0 and float(x[1][0][7]) == 0 and float(x[1][0][8]) == 0 and float(x[1][0][9]) == 0)
reduced1 = filtered1.map(lambda x: ((x[0][0]), 1)).reduceByKey(lambda x, y: x+y)
joined = reduced.join(reduced1)
result = joined.map(lambda x: ((x[0]), ((x[1][1]/x[1][0])*100)))
result1 = result.map(lambda x: (x[0], str(x[1]) + '%'))
result1.saveAsTextFile("task3c.out")

[('024BE2DFD1B98AF1EA941DEDA63A15CB', '1.2738853503184715%'),
 ('03440BEC1C3E0DA656A6091A3CF58261', '0.7299270072992701%'),
 ('096ACCFC956CC07673EC193B24DAAA3E', '2.6548672566371683%'),
 ('0B84045AE4809A95A32F796400C8B403', '0.7246376811594203%'),
 ('0D0D2A78F5C539728E2DCAD4FD6B8AD8', '0.8771929824561403%'),
 ('0DF80FDB0E48E1A2534EE0F509BC66FB', '0.5988023952095809%'),
 ('0E3841C1B577163D9741C5D01354711A', '0.7407407407407408%'),
 ('1087DFA00B5B8DBBDEFD0D32D5E8E0C3', '4.411764705882353%'),
 ('129EB1CA344C038A54D157CB14D346FA', '0.6578947368421052%'),
 ('150C6FAFC492F855EB78C13224D0A3D2', '0.8849557522123894%')]

In [90]:
filtered = spark.sql("select hack_license, medallion from table4 group by hack_license, medallion")
filtered.createOrReplaceTempView("table8")
result = spark.sql("select hack_license, count(medallion) as num_taxis_used from table8 group by hack_license")
output = result.select(format_string('%s, %s', result.hack_license, result.num_taxis_used).alias("result"))
output.write.save("task3d-sql.out", format="text")

+-----------------------------------+
|result                             |
+-----------------------------------+
|69996930170E51265187F2D360A2366D, 7|
|44D39A75B5BADD81E99EFE7B99011E4C, 1|
|3F0CD6765AAD1E5834B1C5AE2C6B3B81, 1|
|6AE65C80B9BE7F24379AAB6F84A2C94E, 7|
|AFD2B319B2455F67B1C59E32921690CF, 1|
|6CDB6B9814FAE69A29CD7082D7798F56, 3|
|5974CCF68C5164D3AB2873F038F5A377, 1|
|A678389B143C4526FE30CA94461BAA2D, 1|
|B5BCB66CBBD83D9D1E84351D9D4B0EA3, 4|
|CE6105A23A0AC634E1199DDAAC13E75D, 5|
|9911D66A4A796752DAA9929262692322, 5|
|A7C47E60941315A0E1B18190584F1B8F, 1|
|F41C758E0A9022F59C72F73D1359FD9A, 3|
|0EB70D3B3BE040DAF5D0E6C15E8119FD, 4|
|594CE5F6F9FBFB980D7699D67F2729F5, 4|
|C5D797C03D817ED5EE0361221C783680, 2|
|DD97899ACAC51EF3188A659DB1F4EDBB, 1|
|EFA12B2F45FC838E19795CA997E26FFA, 1|
|D4F2AE0988ECB2E421AAC0C876483801, 1|
|1700002963A49DA13542E0726B7BB758, 1|
+-----------------------------------+
only showing top 20 rows



In [91]:
mapped = sorted.map(lambda x: ((x[0][1], x[0][0]), 1)).groupByKey()
mapped2 = mapped.map(lambda x: ((x[0][0]), x[0][1]))
result = mapped2.map(lambda x: ((x[0]), 1)).reduceByKey(lambda x, y: x+y).sortByKey()
result.saveAsTextFile("task3d.out")

[('0002555BBE359440D6CEB34B699D3932', 1),
 ('0006C8F9279EFD18D8E70193D98499CB', 1),
 ('0008B3E338CE8C3377E071A4D80D3694', 5),
 ('000A4EBF1CEB9C6BD9978D4362493C6E', 1),
 ('000B8D660A329BBDBF888500E4BD8B98', 4),
 ('000C15EFBDF58CAED528C09E597484EC', 7),
 ('000CCA239BFDC0ABE2895AC9086C4290', 1),
 ('0011B1575B9F5398BBC0F27EA560D631', 3),
 ('0012175D1505490267F025EAD9416CDD', 1),
 ('0012703023AC1788D34F6694908900FC', 1)]

In [95]:
result = spark.sql("select vehicle_type, count(pickup_datetime) as total_trips, sum(fare_amount) as total_revenue, sum(tip_amount/fare_amount) as avg_tip_percentage from table5 group by vehicle_Type")
result.createOrReplaceTempView('table9')
result1 = spark.sql("select vehicle_type, total_trips, cast(total_revenue as DECIMAL(10,2)), cast((avg_tip_percentage * 100)/total_trips as DECIMAL(4,2)) as avg_tip_percentage from table9")
output = result1.select(format_string('%s, %s, %s, %s%%', result1.vehicle_type, result1.total_trips, result1.total_revenue, result1.avg_tip_percentage).alias("result"))
output.write.save("task4a-sql.out", format="text")

+---------------------------------+
|result                           |
+---------------------------------+
|NRML, 75094, 933208.19, 11.01%   |
|HYB, 1523653, 18827047.15, 11.84%|
|LV1, 289979, 3565508.47, 11.03%  |
|CNG, 110224, 1356108.18, 11.12%  |
|DSE, 155542, 1908022.79, 11.02%  |
|WAV, 138641, 1714587.58, 11.02%  |
+---------------------------------+



In [101]:
mapped = sorted1.map(lambda x: ((x[1][1][5]), (x[1][0][2], x[1][0][4], x[1][0][7])))
result = mapped.filter(lambda x: float(x[1][1]) > 0).map(lambda x: ((x[0]), (1, x[1][1], float(x[1][2])/float(x[1][1]))))
result1 = result.reduceByKey(lambda x, y: (x[0] + y[0], float(x[1]) + float(y[1]), x[2] + y[2]))
result2 = result1.map(lambda x: ((x[0]), (x[1][0], x[1][1], (x[1][2]*100)/x[1][0])))
final_result = result2.map(lambda x: (x[0], x[1][0], x[1][1], str(x[1][2]) + '%'))
final_result.saveAsTextFile("task4a.out")

[('HYB', 1522776, 18829377.4, '11.841343700112764%'),
 ('LV1', 289857, 3566022.9699999997, '11.036699201835782%'),
 ('WAV', 138578, 1714587.5799999998, '11.026264390729738%'),
 ('DSE', 155456, 1908340.79, '11.022648539991012%'),
 ('CNG', 110172, 1356270.1800000002, '11.127481353127704%'),
 ('NRML', 75053, 933374.6900000001, '11.018312634174759%')]

In [102]:
result = spark.sql("select medallion_type, count(pickup_datetime) as total_trips, sum(fare_amount) as total_revenue, sum(tip_amount/fare_amount) as avg_tip_percentage from table5 group by medallion_type")
result.createOrReplaceTempView('table10')
result1 = spark.sql("select medallion_type, total_trips, cast(total_revenue as DECIMAL(10,2)), cast((avg_tip_percentage * 100)/total_trips as DECIMAL(4,2)) as avg_tip_percentage from table10")
output = result1.select(format_string('%s, %s, %s, %s%%', result1.medallion_type, result1.total_trips, result1.total_revenue, result1.avg_tip_percentage).alias("result"))
output.write.save("task4b-sql.out", format="text")

+--------------------------------------------+
|result                                      |
+--------------------------------------------+
|NAMED DRIVER, 1610809, 19910259.14, 11.14%  |
|OWNER MUST DRIVE, 682324, 8394223.22, 12.57%|
+--------------------------------------------+



In [105]:
mapped = sorted1.map(lambda x: ((x[1][1][7]), (x[1][0][2], x[1][0][4], x[1][0][7])))
mapped1 = mapped.filter(lambda x: float(x[1][1]) > 0).map(lambda x: ((x[0]), (1, x[1][1], float(x[1][2])/float(x[1][1]))))
result = mapped1.reduceByKey(lambda x, y: (x[0] + y[0], float(x[1]) + float(y[1]), x[2] + y[2]))
result1 = result.map(lambda x: ((x[0]), (x[1][0], x[1][1], (x[1][2]*100)/x[1][0])))
final_result = result1.map(lambda x: (x[0], x[1][0], x[1][1], str(x[1][2]) + '%'))
final_result.saveAsTextFile("task4b.out")

[('OWNER MUST DRIVE', 682057, 8394533.72, '12.57156221334995%'),
 ('NAMED DRIVER', 1609835, 19913439.890000004, '11.150636753592384%')]

In [106]:
filtered = spark.sql("select agent_name, cast(sum(fare_amount) as DECIMAL(9,2)) as total_revenue from table5 group by agent_name order by total_revenue desc, agent_name limit 10")
output = filtered.select(format_string('%s, %s', filtered.agent_name, filtered.total_revenue).alias("result"))
output.write.save("task4c-sql.out", format="text")

+----------------------------------------+
|result                                  |
+----------------------------------------+
|TIRU CABS , 9133120.05                  |
|ALL TAXI MANAGEMENT INC, 1672496.60     |
|TAXIFLEET MANAGEMENT LLC, 1356816.24    |
|GOTHAM YELLOW LLC, 982246.11            |
|TEAM SYSTEMS CORP, 797958.41            |
|WOODSIDE MANAGEMENT INC., 770086.90     |
|OLA CABS, 764637.16                     |
|WHITE AND BLUE GROUP CORP., 666874.38   |
|MC GUINNESS MANAGEMENT CORP, 652309.60  |
|EXECUTIVE OWNERS HOLDING CORP, 617902.69|
+----------------------------------------+



In [109]:
output = sorted1.map(lambda x: ((x[1][1][9]), (x[1][0][4])))
filtered = output.filter(lambda x: float(x[1]) > 0).map(lambda x: ((x[0]), x[1]))
result = filtered.reduceByKey(lambda x, y: (float(x) + float(y)))
result1 = result.map(lambda x: ((x[0]), x[1]))
final_result = result1.sortBy((lambda x: x[1]), False)
top = final_result.take(10)
data = sc.parallelize(top)
data.saveAsTextFile("task4c.out")

[('TIRU CABS ', 9133726.8),
 ('ALL TAXI MANAGEMENT INC', 1672914.6000000003),
 ('TAXIFLEET MANAGEMENT LLC', 1357119.24),
 ('GOTHAM YELLOW LLC', 982382.61),
 ('TEAM SYSTEMS CORP', 798033.91),
 ('WOODSIDE MANAGEMENT INC.', 770133.9000000001),
 ('OLA CABS', 764641.16),
 ('WHITE AND BLUE GROUP CORP.', 667061.8800000001),
 ('MC GUINNESS MANAGEMENT CORP', 652367.1),
 ('EXECUTIVE OWNERS HOLDING CORP', 617928.19)]